In [ ]:
import pandas as pd
import numpy as np
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

from keras.models import Sequential, Model
from keras.layers import Dense,Embedding,LSTM,GRU,Flatten,Dropout, Input, Activation
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.layers.convolutional import Conv1D,MaxPooling1D
from keras.layers import Concatenate, Activation
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
%load_ext tensorboard

In [ ]:
# df_train_full = pd.read_csv("drive/MyDrive/NNDL_Project/train_YELP.csv", header=None)
# df_test_full = pd.read_csv("drive/MyDrive/NNDL_Project/test_YELP.csv", header=None)
# df_train_full.columns = ['label', 'review']
# df_test_full.columns = ['label', 'review']
# df_full = pd.concat([df_train_full.sample(127500), df_test_full.sample(22500)])
# df_full.index = np.arange(len(df_full))

# X = df_full['review']
# y = df_full['label']

# xtr , xte , ytr, yte = train_test_split(X, y, test_size=0.3, random_state=1,stratify=y)
# xte, xval, yte, yval = train_test_split(xte, yte, test_size=0.5, random_state=1, stratify=yte)

# train = pd.DataFrame([xtr, ytr]).T
# train.columns = ['review', 'label']
# train.index = np.arange(len(train))

# val = pd.DataFrame([xval, yval]).T
# val.columns = ['review', 'label']
# val.index = np.arange(len(val))

# test = pd.DataFrame([xte, yte]).T
# test.columns = ['review', 'label']
# test.index = np.arange(len(test))

# train.to_csv('drive/MyDrive/NNDL_Project/YELP_TRAIN_105K.csv', index=False)
# val.to_csv('drive/MyDrive/NNDL_Project/YELP_VAL_22.5K.csv', index=False)
# test.to_csv('drive/MyDrive/NNDL_Project/YELP_TEST_22.5K.csv', index=False)

In [ ]:
df_train = pd.read_csv("drive/MyDrive/NNDL_Datasets/YELP_train.csv")
df_val = pd.read_csv("drive/MyDrive/NNDL_Datasets/YELP_val.csv")
df_test = pd.read_csv("drive/MyDrive/NNDL_Datasets/YELP_test.csv")

In [ ]:
len(df_train), len(df_val), len(df_test)

(105000, 22500, 22500)

In [ ]:
# df_train.columns = ['review', 'label']
# df_val.columns = ['review', 'label']
# df_test.columns = ['review', 'label']

# df_train.to_csv('drive/MyDrive/NNDL_Datasets/YELP_train.csv', index=False)
# df_val.to_csv('drive/MyDrive/NNDL_Datasets/YELP_val.csv', index=False)
# df_test.to_csv('drive/MyDrive/NNDL_Datasets/YELP_test.csv', index=False)

# Text Preprocessing

In [ ]:
stop_words = set(stopwords.words("english"))

def text_preprocessing(dataframe):

  lines = dataframe["review"].values.tolist()
  reviews = list()

  for line in lines:
      tokens = word_tokenize(line)
      tokens = [w.lower() for w in tokens]
      table = str.maketrans("","",string.punctuation)
      stripped = [w.translate(table) for w in tokens]
      words = [w for w in stripped if w.isalpha()]
      words = [w for w in words if w not in stop_words]
      reviews.append(words)

  reviews = [' '.join(review) for review in reviews]
  dataframe['prepro_rev'] = reviews
  return dataframe

In [ ]:
df_train = text_preprocessing(df_train)
df_val = text_preprocessing(df_val)
df_test = text_preprocessing(df_test)

train_text = df_train['prepro_rev'].values
val_text = df_val['prepro_rev'].values
test_text = df_test['prepro_rev'].values

train_text = [s.lower() for s in train_text]
val_text = [s.lower() for s in val_text]
test_text = [s.lower() for s in test_text]

train_classes = df_train['label'].values - 1
val_classes = df_val['label'].values - 1
test_classes = df_test['label'].values - 1

### Tokenizer for Base Model

In [ ]:
EMBED_DIM_WORD = 768
MAX_NUM_WORDS_WORD = 30522
MAX_SEQUENCE_LENGTH_WORD = 400
do_early_stopping = True
VOCAB_SIZE_WORD = MAX_NUM_WORDS_WORD+1

In [ ]:
tokenizer1 = Tokenizer(num_words = MAX_NUM_WORDS_WORD+1, oov_token='UNK')
tokenizer1.fit_on_texts(train_text)

In [ ]:
def custom_text_to_word_sequence(texts, modelTokenizer):
  vects = []
  for text in texts:
    seq = text.split(' ')
    seq = [i for i in seq if i]

    vect = []
    for w in seq:
      try:
        ind = modelTokenizer.word_index[w]
        if (ind<=MAX_NUM_WORDS_WORD):
          vect.append(ind)
        else:
          vect.append(modelTokenizer.word_index['UNK'])
      except KeyError:
        vect.append(modelTokenizer.word_index['UNK'])

    vects.append(vect)
  
  return vects

In [ ]:
x_train_tokens1 = custom_text_to_word_sequence(train_text, tokenizer1)
x_val_tokens1 = custom_text_to_word_sequence(val_text, tokenizer1)
x_test_tokens1 = custom_text_to_word_sequence(test_text, tokenizer1)

x_train_pad1 = pad_sequences(x_train_tokens1,maxlen=MAX_SEQUENCE_LENGTH_WORD,padding="post")
x_val_pad1 = pad_sequences(x_val_tokens1, maxlen=MAX_SEQUENCE_LENGTH_WORD, padding='post')
x_test_pad1 = pad_sequences(x_test_tokens1,maxlen=MAX_SEQUENCE_LENGTH_WORD,padding="post")

### Tokenizer for Character Level Model

In [ ]:
MAX_SEQUENCE_LENGTH_CHAR = 1014

In [ ]:
tokenizer2 = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tokenizer2.fit_on_texts(train_text)

In [ ]:
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

tokenizer2.word_index = char_dict.copy()
tokenizer2.word_index[tokenizer2.oov_token] = max(char_dict.values()) + 1

In [ ]:
x_train_tokens2 = tokenizer2.texts_to_sequences(train_text)
x_val_tokens2 = tokenizer2.texts_to_sequences(val_text)
x_test_tokens2 = tokenizer2.texts_to_sequences(test_text)

x_train_pad2 = pad_sequences(x_train_tokens2,maxlen=MAX_SEQUENCE_LENGTH_CHAR,padding="post")
x_val_pad2 = pad_sequences(x_val_tokens2, maxlen=MAX_SEQUENCE_LENGTH_CHAR, padding='post')
x_test_pad2 = pad_sequences(x_test_tokens2,maxlen=MAX_SEQUENCE_LENGTH_CHAR,padding="post")

# Baseline CNN

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH_WORD,), dtype='int32')
embedding_layer = Embedding(VOCAB_SIZE_WORD, EMBED_DIM_WORD, input_length=MAX_SEQUENCE_LENGTH_WORD)

input_node = Input(shape=(MAX_SEQUENCE_LENGTH_WORD, EMBED_DIM_WORD))
conv_list = []

conv = Conv1D(filters=10, kernel_size=3, activation='relu')(input_node)
drop = Dropout(0.3)(conv)
pool = MaxPooling1D(pool_size=2)(drop)
flatten = Flatten()(pool)
conv_list.append(flatten)

conv = Conv1D(filters=10, kernel_size=8, activation='relu')(input_node)
drop = Dropout(0.3)(conv)
pool = MaxPooling1D(pool_size=2)(drop)
flatten = Flatten()(pool)
conv_list.append(flatten)

out = Concatenate()(conv_list)
graph = Model(inputs = input_node, outputs = out)

model1 = Sequential()
model1.add(embedding_layer)
model1.add(Dropout(0.5, input_shape=(MAX_SEQUENCE_LENGTH_WORD, EMBED_DIM_WORD)))
model1.add(graph)
model1.add(Dense(50))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(1, activation='sigmoid'))

optimizer = optimizers.Adam(learning_rate=0.0004)

model1.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 768)          23441664  
                                                                 
 dropout_2 (Dropout)         (None, 400, 768)          0         
                                                                 
 model (Functional)          (None, 3950)              84500     
                                                                 
 dense (Dense)               (None, 50)                197550    
                                                                 
 activation (Activation)     (None, 50)                0         
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 5

In [ ]:
tensorboard1 = TensorBoard(log_dir='drive/MyDrive/NNDL_Project/Dataset2/CNN_Baseline_logs/', histogram_freq=0, write_graph=True)
early_stopping1 = EarlyStopping(monitor='val_loss', patience = 2, mode = 'min')
cp1 = ModelCheckpoint('drive/MyDrive/NNDL_Project/Dataset2/CNN_Baseline_bestModel.h5', monitor='val_acc', save_best_only=True, mode='max')

print('using early stopping strategy')
history1 = model1.fit(x_train_pad1, train_classes, validation_data=(x_val_pad1, val_classes), epochs=5, batch_size=16, callbacks = [early_stopping1, cp1, tensorboard1])

using early stopping strategy
Epoch 1/5
6563/6563 [==============================] - 280s 38ms/step - loss: 0.2486 - acc: 0.8948 - val_loss: 0.1950 - val_acc: 0.9251
Epoch 2/5
6563/6563 [==============================] - 251s 38ms/step - loss: 0.1573 - acc: 0.9407 - val_loss: 0.1876 - val_acc: 0.9268
Epoch 3/5
6563/6563 [==============================] - 245s 37ms/step - loss: 0.1153 - acc: 0.9580 - val_loss: 0.2005 - val_acc: 0.9251
Epoch 4/5
6563/6563 [==============================] - 245s 37ms/step - loss: 0.0791 - acc: 0.9718 - val_loss: 0.2284 - val_acc: 0.9190


In [ ]:
results1 = model1.evaluate(x_test_pad1, test_classes, batch_size=16)

1407/1407 [==============================] - 9s 7ms/step - loss: 0.2204 - acc: 0.9224


# Character Level CNN

In [ ]:
EMBED_DIM_CHAR, VOCAB_SIZE_CHAR

(69, 70)

In [ ]:
conv_layers_sm = [[256, 7, 3],
                  [256, 7, 3],
                  [256, 3, -1],
                  [256, 3, -1],
                  [256, 3, -1],
                  [256, 3, 3]]

fully_connected_layers_sm = [1024, 1024]

num_of_classes = 2
dropout_p = 0.5
optimizer = 'adam'
loss = 'binary_crossentropy'
EMBED_DIM_CHAR = len(tokenizer2.word_index)
VOCAB_SIZE_CHAR = EMBED_DIM_CHAR + 1

In [ ]:
# Model Construction
# Input
inputs = Input(shape=(MAX_SEQUENCE_LENGTH_CHAR,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding
x = Embedding(VOCAB_SIZE_CHAR,
              EMBED_DIM_CHAR,
              input_length=MAX_SEQUENCE_LENGTH_CHAR,)(inputs)
# Conv
for filter_num, filter_size, pooling_size in conv_layers_sm:
    x = Conv1D(filter_num, filter_size)(x)
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
x = Flatten()(x)  # (None, 8704)
# Fully connected layers
for dense_size in fully_connected_layers_sm:
    x = Dense(dense_size, activation='relu')(x)  # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(1, activation='sigmoid')(x)
# Build model
model2 = Model(inputs=inputs, outputs=predictions)
model2.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])  # Adam, binary_crossentropy
print(model2.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 1014)]            0         
                                                                 
 embedding_1 (Embedding)     (None, 1014, 69)          4830      
                                                                 
 conv1d_2 (Conv1D)           (None, 1008, 256)         123904    
                                                                 
 activation_1 (Activation)   (None, 1008, 256)         0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 336, 256)         0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 330, 256)          459008    
                                                           

In [ ]:
tensorboard2 = TensorBoard(log_dir='drive/MyDrive/NNDL_Project/Dataset2/CNN_Char_sm_logs/', histogram_freq=0, write_graph=True)
early_stopping2 = EarlyStopping(monitor='val_loss', patience = 2, mode = 'min')
cp2 = ModelCheckpoint('drive/MyDrive/NNDL_Project/Dataset2/CNN_Char_sm_bestModel.h5', monitor='val_acc', save_best_only=True, mode='max')

print('using early stopping strategy')
history2 = model2.fit(x_train_pad2, train_classes, validation_data=(x_val_pad2, val_classes), epochs=15, batch_size=128, callbacks = [early_stopping2, cp2, tensorboard2])

using early stopping strategy
Epoch 1/15
821/821 [==============================] - 207s 214ms/step - loss: 0.4364 - accuracy: 0.7772 - val_loss: 0.2992 - val_accuracy: 0.8668
Epoch 2/15
821/821 [==============================] - 164s 200ms/step - loss: 0.2542 - accuracy: 0.8947 - val_loss: 0.2288 - val_accuracy: 0.9060
Epoch 3/15
821/821 [==============================] - 164s 199ms/step - loss: 0.2106 - accuracy: 0.9146 - val_loss: 0.2574 - val_accuracy: 0.8956
Epoch 4/15
821/821 [==============================] - 164s 199ms/step - loss: 0.1812 - accuracy: 0.9274 - val_loss: 0.2336 - val_accuracy: 0.9031


In [ ]:
results2 = model2.evaluate(x_test_pad2, test_classes, batch_size=128)

176/176 [==============================] - 11s 61ms/step - loss: 0.2293 - accuracy: 0.9055
